In [ ]:
import pandas as pd
import torch
import torch.nn as nn
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import DataLoader, TensorDataset

In [ ]:
train_data = pd.read_excel('train.xlsx')

y = train_data.iloc[:, 0].values.reshape(-1, 1)
X = train_data.iloc[:, 1:6].values

scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()

X_scaled = scaler_X.fit_transform(X)
y_scaled = scaler_y.fit_transform(y)

X_tensor = torch.tensor(X_scaled, dtype=torch.float32)
y_tensor = torch.tensor(y_scaled, dtype=torch.float32)

train_dataset = TensorDataset(X_tensor, y_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

In [ ]:
class TransformerModel(nn.Module):
    def __init__(self, input_dim, output_dim, nhead=1, num_layers=2, dim_feedforward=128):
        super(TransformerModel, self).__init__()
      
        d_model = input_dim
        if d_model % nhead != 0:
            raise ValueError("input_dim must be nhead")
        
        self.transformer = nn.Transformer(
            d_model=d_model,
            nhead=nhead,
            num_encoder_layers=num_layers,
            num_decoder_layers=num_layers,
            dim_feedforward=dim_feedforward
        )
        
        self.fc = nn.Linear(input_dim, output_dim)

    def forward(self, src):
        src = src.unsqueeze(1)  
        output = self.transformer(src, src)
        output = output.mean(dim=3) 
        output = self.fc(output)
        return output

In [ ]:
input_dim = X_tensor.shape[1]
output_dim = y_tensor.shape[1]

model = TransformerModel(input_dim, output_dim, nhead=1)

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.005)

epochs = 200
for epoch in range(epochs):
    model.train()
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        output = model(X_batch)
        loss = criterion(output, y_batch)
        loss.backward()
        optimizer.step()

    if epoch % 10 == 0:
        print(f'Epoch {epoch}, Loss: {loss.item()}')

In [ ]:
test_data = pd.read_excel('test.xlsx')
X_test = test_data.iloc[:, :].values
X_test_scaled = scaler_X.transform(X_test)
X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)

model.eval()
with torch.no_grad():
    y_pred_scaled = model(X_test_tensor).numpy()

y_pred = scaler_y.inverse_transform(y_pred_scaled)

pred_df = pd.DataFrame(y_pred, columns=['prediction'])
pred_df.to_excel('result.xlsx', index=False)


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

train_data = pd.read_excel('train.xlsx')
y_train = train_data.iloc[:, 0].values  
X_train = train_data.iloc[:, 1:6].values  

scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()

X_train_normalized = scaler_X.fit_transform(X_train)
y_train_normalized = scaler_y.fit_transform(y_train.reshape(-1, 1))

X_train_lstm = X_train_normalized.reshape((X_train_normalized.shape[0], 1, X_train_normalized.shape[1]))

lstm_model = Sequential()
lstm_model.add(LSTM(50, input_shape=(X_train_lstm.shape[1], X_train_lstm.shape[2])))
lstm_model.add(Dense(3))
lstm_model.compile(optimizer='adam', loss='mse')

lstm_model.fit(X_train_lstm, y_train_normalized, epochs=300, batch_size=20, verbose=1)

test_data = pd.read_excel('test.xlsx')
X_test = test_data.values

X_test_normalized = scaler_X.transform(X_test)
X_test_lstm = X_test_normalized.reshape((X_test_normalized.shape[0], 1, X_test_normalized.shape[1]))

y_pred_normalized = lstm_model.predict(X_test_lstm)
y_pred = scaler_y.inverse_transform(y_pred_normalized)

result_df = pd.DataFrame(y_pred, columns=['Predicted Wind Power'])
result_df.to_excel('lstm.xlsx', index=False)

In [ ]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.neural_network import MLPRegressor

train_data = pd.read_excel('train.xlsx')
y_train = train_data.iloc[:, 0].values     
X_train = train_data.iloc[:, 1:6].values      

scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()

X_train_normalized = scaler_X.fit_transform(X_train) 
y_train_normalized = scaler_y.fit_transform(y_train.reshape(-1, 1)).ravel() 

model = MLPRegressor(
    hidden_layer_sizes=(128, 64), 
    activation='relu',
    solver='adam',
    learning_rate_init=0.001,
    max_iter=2000,
    random_state=42,
    early_stopping=True,
    n_iter_no_change=20
)

model.fit(X_train_normalized, y_train_normalized)

test_data = pd.read_excel('test.xlsx')
X_test = test_data.values 

X_test_normalized = scaler_X.transform(X_test)

y_pred_normalized = model.predict(X_test_normalized)
y_pred = scaler_y.inverse_transform(y_pred_normalized.reshape(-1, 1))

result_df = pd.DataFrame(y_pred, columns=['Predicted Wind Power'])
result_df.to_excel('mlp.xlsx', index=False)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense

train_data = pd.read_excel('train.xlsx')
y_train = train_data.iloc[:, 0].values  
X_train = train_data.iloc[:, 1:6].values  

scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()

X_train_normalized = scaler_X.fit_transform(X_train)
y_train_normalized = scaler_y.fit_transform(y_train.reshape(-1, 1))

X_train_cnn = X_train_normalized.reshape((X_train_normalized.shape[0], X_train_normalized.shape[1], 1))

cnn_model = Sequential()
cnn_model.add(Conv1D(64, kernel_size=2, activation='relu', input_shape=(X_train_cnn.shape[1], 1)))
cnn_model.add(MaxPooling1D(pool_size=2))
cnn_model.add(Flatten())
cnn_model.add(Dense(50, activation='relu'))
cnn_model.add(Dense(50, activation='relu'))
cnn_model.add(Dense(3))
cnn_model.compile(optimizer='adam', loss='mse')

cnn_model.fit(X_train_cnn, y_train_normalized, epochs=200, batch_size=20, verbose=1)

test_data = pd.read_excel('test.xlsx')
X_test = test_data.values

X_test_normalized = scaler_X.transform(X_test)
X_test_cnn = X_test_normalized.reshape((X_test_normalized.shape[0], X_test_normalized.shape[1], 1))

y_pred_normalized = cnn_model.predict(X_test_cnn)
y_pred = scaler_y.inverse_transform(y_pred_normalized)

result_df = pd.DataFrame(y_pred, columns=['Predicted Wind Power'])
result_df.to_excel('cnn.xlsx', index=False)